# Web Scrapping

In [1]:
from bs4 import BeautifulSoup
import requests
import os
import PyPDF2 

In [699]:
def get_job_pages(url_search,url_base):
    """ 
    Return a list of links to jobs returned from the url_search link
    """
    # use requests to get the search url contents, convert to a soup object
    r = requests.get(url_search)
    soup = BeautifulSoup(r.content)
    
    # extract the links from the job titles
    pages = soup.find_all("a",{"class":"Pagination-link"})    
    
    #     url_search+"&pn=2"
    
    page_links = []
    for page in pages:
        p = page.get('href')
        page_links.append(f"{url_base}{p}")
        
    
    return set(page_links)
get_job_pages(url_search, url_base)

{'https://www.workopolis.com/jobsearch/find-jobs?ak=data%20science%20-intern%20-co-op&l=Toronto%2C%20ON&sr=15&t=-1&jt=fulltime&job=yIS2ImDBq4vHKMdNQREsuoJsHwkenk-JmN4tv3z4ltkgfzHOH7-k1tjdI-bnlXM2&pc=ABkAA4EASwAAAAAAAAAAAAAAAgAw5RAAMQEBARAAxtVUDuMsFBSEJ6wCIxU6O23sq4753JcWHFT%2FSJ3KlXoC0SGlbMw3DhVkKVgAAA%3D%3D&pn=4',
 'https://www.workopolis.com/jobsearch/find-jobs?ak=data%20science%20-intern%20-co-op&l=Toronto%2C%20ON&sr=15&t=-1&jt=fulltime&job=yIS2ImDBq4vHKMdNQREsuoJsHwkenk-JmN4tv3z4ltkgfzHOH7-k1tjdI-bnlXM2&pc=ABkAAYEAGQAAAAAAAAAAAAAAAgAw5RAAJgEBAQcwsREyVvzzoy82MJ0jj1Y7yXDE6y%2BRF6VHol3YJUKYaQg5AAA%3D&pn=2',
 'https://www.workopolis.com/jobsearch/find-jobs?ak=data%20science%20-intern%20-co-op&l=Toronto%2C%20ON&sr=15&t=-1&jt=fulltime&job=yIS2ImDBq4vHKMdNQREsuoJsHwkenk-JmN4tv3z4ltkgfzHOH7-k1tjdI-bnlXM2&pc=ABkAAoEAMgAAAAAAAAAAAAAAAgAw5RAAOAECAQ0wBgIv%2BadxA6UjIIyvJSxym%2BJlT2FOMW0OgR23GjIwRriMg7avPqc%2Fs3u8Hmumcp1Bz72TAAA%3D&pn=3',
 'https://www.workopolis.com/jobsearch/find-jobs?ak=data%

In [697]:
def get_job_links(url_search,url_base):
    """ 
    Return a list of links to jobs returned from the url_search link
    """
    # use requests to get the search url contents, convert to a soup object
    r = requests.get(url_search)
    soup = BeautifulSoup(r.content)
    
    # extract the links from the job titles
    jobs = soup.find_all("a",{"class":"SerpJob-titleLink"})    
    
    # return a list of job-links
    job_links = []
    for job in jobs:
        link = f"{url_base}{job.get('href')}"
        job_links.append(link)
    return job_links

In [696]:
job_links

['https://www.workopolis.com/jobsearch/viewjob/nA6YQb9WK7tJZTWVwO8XMYS3kjg1Rk2CHqcdJzOpt4zg7jNPbjMI8XwnSkh3OUOz',
 'https://www.workopolis.com/jobsearch/viewjob/YR7b5pmjzWLK4xk538329tdm1HqirGUSRAm3pRsXsNxKvLWqesftXBJLrk-vxe9s',
 'https://www.workopolis.com/jobsearch/viewjob/1YFHfv85JRc7u9IeYfpOqFe4rFVBDybSmZUqPpTagCWZMIcbATz6uQhy2MMATEf_',
 'https://www.workopolis.com/jobsearch/viewjob/i-d6E8G0buWRjSJcxj_cGDnw-MaAtR0RXIuNbkhZFlJUb8vC4orhJq1Ky2pNwZpX',
 'https://www.workopolis.com/jobsearch/viewjob/nxGrTWBAT_GJARC4_GL41wEKDBnckjMvO2t88OHibrSba05L78eU3dXGqRfz3Vqt',
 'https://www.workopolis.com/jobsearch/viewjob/ienXxamslyGsLUpKAkICiYQmi8uyWXXx2bVburOQbabQy-Rg-sCzDIsTYrK5eUjt',
 'https://www.workopolis.com/jobsearch/viewjob/Lwmk1pgFExJXYNf1yzbnn2RUr-EpyTWerAwX9fG1eiJZ5of02-y7zMur-A18IZAR',
 'https://www.workopolis.com/jobsearch/viewjob/IjRAqTOf-z0tv45uVDBHiGYKx5-gBH9X6IdXipzy2G36wwjopo3aCS4c1DsSpwjM',
 'https://www.workopolis.com/jobsearch/viewjob/7i50TvroZHbEEVlEf35KCgM7JcqnjT8u1-YdzHK2n

In [673]:
def get_job_details(urls):
    """
    """
    print("Scrapping RSS Job Search Feed")
    print("="*100)
    job_detail = []
    for url in urls:
        try:
            # get url content, convert to soup
            r = requests.get(url)
            soup = BeautifulSoup(r.content)

            # extract content
            title = soup.find("div",{'class':'ViewJobHeaderTitle'}).text
            company = soup.find("div",{'class':'ViewJobHeaderCompany'}).text
            location = soup.find("span",{'class':'ViewJobHeaderPropertiesLocation'}).text
            desc = soup.find("div",{'class':'viewjob-description'}).text


            # extract key words (skills, education, benefits)
            keys = soup.find_all('li',{'class':'nav-item'})
            key_words = [w.text for w in keys]

            # build result-set
            job_detail.append(
                {'title':title,'company':company,'location':location,'desc':desc,'keywords':key_words}
            )
            # print progress
            print(f"{title[:18]:{18}}| {company[:15]:{15}}| {location[:7]:{7}} | {key_words[:3]}...")
        except Exception as e:
            print(f"   =>Skipping Job...{e}")
            continue
    print("="*100)
    return job_detail

In [694]:
url_search+"&pn=20"

'https://www.workopolis.com/jobsearch/find-jobs?ak=data%20science%20-intern%20-co-op&l=Toronto,%20ON&sr=15&t=-1&jt=fulltime&job=yIS2ImDBq4vHKMdNQREsuoJsHwkenk-JmN4tv3z4ltkgfzHOH7-k1tjdI-bnlXM2&pn=20'

In [679]:

url_search = 'https://www.workopolis.com/jobsearch/find-jobs?l=Toronto%2C%20ON&ak=data%20science%20-intern&t=7&sr=10&s=r&job=zD_1ItN-5JXVPUCwXgZvg_joKB7qyQNp5ttncJBtECQ1FoqVIwMR8znJuj3fF7lK'
url_search = 'https://www.workopolis.com/jobsearch/find-jobs?ak=data%20science%20-intern%20-co-op&l=Toronto,%20ON&sr=15&t=-1&jt=fulltime&job=yIS2ImDBq4vHKMdNQREsuoJsHwkenk-JmN4tv3z4ltkgfzHOH7-k1tjdI-bnlXM2'
url_base  ='https://www.workopolis.com'

job_links = get_job_links(url_search, url_base)
job_details = get_job_details(job_links)


Scrapping RSS Job Search Feed
Data Scientist    | Wish           | Toronto | ["Master's degree", "Bachelor's", 'MongoDB']...
Data Analyst      | WGames Inc.    | Toronto | ["Bachelor's", 'R', 'Google Cloud Platform']...
Data Analyst - Ope| Flair Airlines | Toronto | ["Bachelor's", 'Power BI', 'Microsoft Excel']...
Data Analyst      | Brik Partners  | Toronto | ["Bachelor's", 'Computer science', 'MATLAB']...
Data Analyst      | Environics Anal| Toronto | ['Post-secondary education', 'Microsoft Powerpoint', 'Microsoft Word']...
Data Analyst      | Connexall      | North Y | ["Bachelor's", 'Microsoft Powerpoint', 'Power BI']...
Data Scientist    | Deep Genomics  | Toronto | ["Master's degree", "Bachelor's", 'R']...
Data scientist    | Autocase       | Toronto | ["Master's degree", "Bachelor's", 'Power BI']...
Data Scientist    | Clearco        | Toronto | ["Bachelor's", 'DevOps', 'SQL']...
Data Scientist - A| Royal Bank of C| Toronto | ['Microsoft Excel', 'Azure', 'Big data']...
NLP Data 

# NLP

In [139]:
# Sumy
import sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
import nltk # required for sumy
nltk.download('punkt') # required for sumy

# spacy
import spacy
from spacy import displacy
NLP = spacy.load('en_core_web_lg')
from spacy.matcher import PhraseMatcher
# from spacy.matcher import Matcher

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


from tabulate import tabulate
from collections import Counter
import docx2txt
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

[nltk_data] Downloading package punkt to /home/ken/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Resume Scrapping

In [634]:
def get_resume_text(path,print_text=True):
    """ 
    Read resume text content from file (pdf or docx)
    Params:
        path: string- path to the resume file
        print_text: boolean.  Prints the text
    Returns: A text object with the resume file content
    """
    
    # open file and get file extension
    file = open(path,'rb')
    file_name, file_ext = os.path.splitext(path)
    
    # extract text content
    if file_ext=='.pdf':
        # read pdf
        pdf = PyPDF2.PdfFileReader(file)
        
        text = """"""
        for page_num in range(pdf.numPages):
            page = pdf.getPage(page_num)
            content = page.extract_text()
            text += content
    elif file_ext=='.docx':
        text = docx2txt.process(file)
    elif file_ext=='.txt':
        text = str(file.read())
        
    if print_text:
        print(text)

    file.close()
    return text

In [8]:
def show_pos(text):    
    """Extracts POS from a spacy document and prints the results"""
    d=[]
    for token in doc:
        d.append({
            "Token":token.text,
            "POS":token.pos_,
            "POS-Desc":spacy.explain(token.pos_),
            "Lemma":token.lemma_,
            "Stop-Word":token.is_stop,
            "POS-Detail":token.tag_,
            "POS-Detail-Desc":spacy.explain(token.tag_)
            }
        )
    print(tabulate(d,headers="keys",tablefmt="github"))

In [9]:
def show_entities(doc,display=True):
    """Print a list of named entities and their descriptions"""
    d=[]
    if doc.ents:
        for entity in doc.ents:
            d.append({
                "Word":entity,
                "Entity":entity.label_,
                "Start":entity.start,
                "Stop":entity.end,
                "Entity-Desc":spacy.explain(entity.label_),
                }
            )
        if display:
            print(tabulate(d,headers="keys",tablefmt="github"))
            print("\n")
            displacy.render(doc,style='ent',jupyter=True)
            print("\n")
    else:
        print("no entities found")
    return d   

In [10]:
def attr_counts(doc,att):
    """Return a table of counts by attributes"""
    if att.lower() == 'pos':
        cnts = doc.count_by(spacy.attrs.POS)
    elif att.lower() =='tag':
        cnts = doc.count_by(spacy.attrs.TAG)
    elif att.lower() =='lemma':
        cnts = doc.count_by(spacy.attrs.LEMMA)
    elif att.lower() =='entity':
        cnts = doc.count_by(spacy.attrs.ENT_TYPE)
    elif att.lower() =='email':
        cnts = doc.count_by(spacy.attrs.LIKE_EMAIL)
    elif att.lower() =='url':
        cnts = doc.count_by(spacy.attrs.LIKE_URL)
    elif att.lower() =='currency':
        cnts = doc.count_by(spacy.attrs.IS_CURRENCY)
    elif att.lower() =='numbers':
        cnts = doc.count_by(spacy.attrs.IS_DIGIT)
        
    # lookup attribute name
    d = {doc.vocab[k].text:v for k,v in cnts.items()}
    return d

### Generate Text Summary

In [502]:
def get_text_summary(text,text_source,summarizer,num_sentences,print_summary=True):
    """
    Generate a text summary from a document (extractive)
    Params:
        text: plain text or path to a document
        text_source: text or file
        summarizer: text-summarization method. lex-rank, text-rank,lsa
        num_sentences: Number of summary sentences to generate
        print_summary: T/F
    Returns:
        A list of sentences.
    """
    # Create parser from text-source
    if text_source=='text':
        parser = PlaintextParser.from_string(text,Tokenizer('english'))
    elif text_source=='file':
        parser = PlaintextParser.from_file(text,Tokenizer('english'))
    else:
        raise Exception(f"text source must be 'text' or 'file'!")
        
    # Select summarizer method
    if summarizer=='lex-rank':
        # Lex Rank:  A graph-based summarization method that uses keyword extractions
        s = LexRankSummarizer()
    elif summarizer=='text-rank':
        # Text rank: A graph-based summarization method that uses keyword extractions
        s = TextRankSummarizer()
    elif summarizer=='lsa':
        # Latent semantic analysis 
        # Combines term frequency with singular value decomposition
        s = LsaSummarizer(Stemmer("english"))
        s.stop_words = get_stop_words("english")
    else:
        raise Exception(f"Summarizer must be 'lex-rank','luhn','text-rank' or 'lsa'!")
    
    # create summary
    summary = s(parser.document,num_sentences)
    
    # print summary
    text = ""
    if print_summary:
        print(f'Text Summary:Top {num_sentences} sentences using a {summarizer} summarizer')
        print('='*100)
        for sentence in summary:
            text += " "+ str(sentence)
            print(sentence)
        print('='*100,'\n')
    return text

### Keyword Analysis

In [12]:
def get_terms(path):
    """
    Read a csv of data science terms & return themn as a list
    Params: path -string
    Returns: A list of terms
    """
    with open(path,'r') as f:
        lst = [line.lower().strip() for line in f.readlines()]
    return set(lst)

In [ ]:
def extract_keywords(text,num, print_words=True):
    """
    Extracts keywords from a text object.  Ignores common job positng phrases
    Params:
        text: string text object
        num: integer.  The number of keywords to extract
        print_words: boolean. Prints the output
    Returns: A list of the top keywords in the text object
    """
    words = []
    pos = ['NOUN','PROPN']
    common_job_keys = ['experience','job','ability','posting','process','date','stakeholders','skills','solutions','project']
    tokens = NLP(text.lower())
    
    for token in tokens:
        if (token.text not in NLP.Defaults.stop_words and token.text.lower() not in common_job_keys and token.pos_ in pos):
            words.append(token.text)
            
    # get the top keywords by frequency
    keyword_frequency = [word for word in Counter(words).most_common(num)]
    keywords = [word[0] for word in keyword_frequency]
    
    if print_words:
        print(keyword_frequency)
    return keywords            

In [467]:
# extract keywords from the job descriptions
keys = []
for job in job_details:
    keys.extend(extract_keywords(job['desc'],50,False))
keys = set(keys)

In [466]:
# extract keywords from the job keywords
keys = []
for job in job_details:
    keys.extend(job['keywords'])
keys = set(keys)

In [77]:
def get_phrases_from_keywords(keywords):
    lst = []
    for keyword in keywords:
        if " " in keyword:
            lst.append(keyword.lower().replace(" ","-"))
        lst.append(keyword.lower())
    return set(lst)

In [206]:
def get_phrase_matches(keywords,text):
    """
    spacy 3.5 has a built in fuzzy matching option
    """
    # when spacy 3.5 is available on anaconda
#     matcher = Matcher(NLP.vocab)
#     patterns = [{"TEXT": {"FUZZY": "modelling"}},{"TEXT": {"FUZZY": "machine learning"}}]

    # get the term list
    terms = get_phrases_from_keywords(keywords)
    
    # convert text to tokens
    doc = NLP(text.lower())

    # phrase matching
    patterns = [NLP(text) for text in terms]

    # add to matcher
    matcher = PhraseMatcher(NLP.vocab)
    matcher.add('KEYWORDS',patterns)

    # search for phrase matches
    found_matches = matcher(doc)

    matched_phrases = []
    for match_id, start, end in found_matches:
        matched_phrases.append(doc[start:end].text)

    return dict(Counter(matched_phrases)) 

In [388]:
def compare_keywords(resume,job,keyword_path):
    """
    """
    # path to csv of terms
    keys = get_phrases_from_keywords(get_terms(keyword_path))
    
    # match phrases
    matches_resume = get_phrase_matches(keys,resume)
    matches_job = get_phrase_matches(keys,job)

    # convert to dataframes
    df_resume = pd.DataFrame.from_dict(matches_resume,orient='index',columns=['resume'])
    df_job = pd.DataFrame.from_dict(matches_job,orient='index',columns=['job'])
    
    # combine results & sort by job keywords
    df = df_job.join(df_resume)
    df.sort_values(by=['job'], ascending=False,inplace=True)
    
    # calculate the weighted difference in keyword frequencies
    df['wt'] = df['job']/df['job'].sum()
    df['wt-score'] = df['wt']*(df['resume']-df['job'])
    score = df['wt-score'].sum()
    
    # calculate keyword coverage ratio
    matches = df.query('~resume.isnull()').shape[0]
    total = df.shape[0]
    coverage = matches/total
    
    # get missing
    missing = set(df.query('resume.isnull()').index)
    
    
    return df, score, coverage, missing

In [245]:
def compare_job_specific_keywords(resume, job_keywords):
    """
    """
    # match phrases
    matches = get_phrase_matches(job_keywords,resume)
    
    # add missing job keywords to the dict
    for key in job_keywords:
        if key.lower() in matches.keys():
            continue
        else:
            matches[key.lower()]=0
    
    # convert to dateframe
    df = pd.DataFrame.from_dict(matches,orient='index',columns=['resume'])
    
    return df

In [454]:
def plot_keyword_comparison(resume, job_details, keyword_path):
    """ 
    Plot Job keywords vs Resume keywords
    Params: df- dataframe output from compare_keywords
    Returns: None
    """
    
    # compare common keywords, split into matches/no-matches
    df, score, coverage, missing = compare_keywords(resume,job_details['desc'],keyword_path)
    df_missing  = df.query('resume.isnull()')
    df_matching = df.query('~resume.isnull()')
    
    title = f"Keyword Coverage={round(coverage*100,2)}% | Weighted Average={round(score,2)}"
    
    # compare to job specific keywords
    df_spec = compare_job_specific_keywords(resume, job_details['keywords'])
    
    col_scale = [
        [0,'rgba(252,76,1,0.9)'],
        [0.25,'rgba(252,76,1,0.5)'],
        [0.5,'rgba(83,128,141,0.25)'],
        [0.75,'rgba(83,128,141,0.5)'],
        [1.0,'rgba(83,128,141,0.9)']
    ]
    
    # create plots
    fig = make_subplots(
        rows=1,
        cols=3,
        subplot_titles = ('Matching Keywords','Missing Keywords','Job Specific Keywords'),
        column_widths = [0.5,0.25,0.25]
    )
    # matching keywords
    fig.add_trace(
        go.Bar(
            name='Weighted Frequency',
            y = df_matching.index,
            x = df_matching['wt-score'],
            orientation='h',
            marker = dict(color=df_matching['wt-score'],colorscale=col_scale)
        ),row=1,col=1
    )
    # missing keywords
    fig.add_trace(
        go.Bar(
            name='Missing Keywords',
            y = df_missing.job,
            x = df_missing.index,
            marker = dict(color='rgba(252,76,1,0.8)')
        ),row=1,col=2
    )
    # job specific keywords
    fig.add_trace(
        go.Bar(
            name='Job Specific Keywords',
            y = df_spec.resume,
            x = df_spec.index,
            marker = dict(color='rgba(252,76,1,0.8)')
        ),row=1,col=3
    )
    
    # adjusments
    fig.update_xaxes(tickangle=90,row=1,col=2)
    fig.update_xaxes(tickangle=90,row=1,col=3)
    fig.update_xaxes(range=[-2,2],row=1,col=1)
    fig.update_xaxes(showgrid=False)
    fig.update_layout(
        template='plotly_white',
        title=title,
        width = 900,
        height=500,
        showlegend=False
    )
    fig.show()

## Calculate Similarities

In [26]:
def preprocess_text(text):
    """
    Preprocess text documents in preparation for analysis
    Params:
        text: a text object
    Returns: 
        A text object after converting to lowercase lemmas, removing stop-words,punctuation, numbers, and emails
    """
    
    # replace carriage returns and convert to lowercase
    doc = text.replace("\n","").replace("  ","").lower()
    
    # tokenize
    tokens = NLP(doc)

    text_list = []
    for token in tokens:
        if token.text in NLP.Defaults.stop_words:
            continue
        if token.is_punct:
            continue
        if token.is_digit:
            continue
        if token.like_email:
            continue
        if token.lemma_=='-PRON-':
            continue
            
        # lemmetize
        text_list.append(token.lemma_)
    return " ".join(text_list)

In [27]:
def calc_similarity(doc1,doc2):
    """ 
    Calculate the cosine similarity between two documents
    Params:
        doc1 - text document 
        doc2 - text document
    Returns: The cosine similairty score between the documents
    """
    d1 = preprocess_text(doc1)
    d2 = preprocess_text(doc2)
    
    X = TfidfVectorizer().fit_transform([d1,d2])
    score = cosine_similarity(X[0],X[1])
    return score[0][0]

In [390]:
def calc_similarities(resume, jobs,keyword_path):
    """ 
    Calculate the cosine similarity between a resume and a list of job descriptions
    Params:
        resume: a resume (text)
        jobs: a dictionary of jobs listings (output from get_job_details)
    Returns: A dataframe of job title, company, location and similarity score
    """
    scores = []
    titles = []
    companies =[]
    locs = []
    key_cov = []
    key_wt = []
    key_missing = []
    for job in jobs:
        titles.append(job['title'])
        companies.append(job['company'])
        locs.append(job['location'])
        scores.append(calc_similarity(resume, job['desc']))
        _,wt,cov,missing = compare_keywords(resume,job['desc'],keyword_path)
        key_cov.append(cov)
        key_wt.append(wt)
        key_missing.append(missing)
    
    data = list(zip(titles,companies,locs, scores, key_cov,key_wt,key_missing ))
    df = pd.DataFrame(data,columns = ['title','company','location','similarity-score','keyword-coverage','keyword-wt','keywords-missing'])
    
    return df.sort_values(by=['similarity-score'], ascending=False)

In [575]:
def plot_similarity_scores(df_similarities):
    """
    """
    import numpy as np
    
    # min-max scale the keyword weights
    df = df_similarities.copy()
    df['keyword-weight'] = (df['keyword-wt']-min(df['keyword-wt']))/(max(df['keyword-wt'])-min(df['keyword-wt']))
    
    # custom colorscale
    col_scale = [
        [0,'rgba(252,76,1,1.0)'],
        [0.25,'rgba(252,76,1,0.5)'],
        [0.5,'rgba(83,128,141,0.25)'],
        [0.75,'rgba(83,128,141,0.5)'],
        [1.0,'rgba(83,128,141,1.0)']
    ]
    
    # custom hover data
    hover_data = np.stack((df['title'],df['company'],df['location'],df['keyword-weight']),axis=-1)
        
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x = df['keyword-coverage'],
            y = df['similarity-score'],
            mode='markers',
            customdata = hover_data,
            hovertemplate = '<b>Title:</b> %{customdata[0]}<br>' +
                            '<b>Company:</b> %{customdata[1]}<br>'+
                            '<b>Location:</b> %{customdata[2]}</br>'+
                            '<b>Similarity:</b> %{y:,.2f}<br>' +
                            '<b>Keyword Coverage:</b> %{x:,.2f}<br>' +
                            '<b>Weighted Keyword Frequency:</b> %{customdata[3]:,.2f}<br>' +
                            '<extra></extra>',
            marker = dict(
                size=df['keyword-weight']*50,
                color = df['similarity-score'],
                colorscale=col_scale
            )
        )
    )
    fig.update_xaxes(title='Keyword Coverge (%)',range=[0,1.0])
    fig.update_yaxes(title='Similarity Score',range=[0,1.0])
    fig.update_layout(
        template='plotly_white',
        title='Resume Similarity to Job Listings',
        width=800,
        height=500
    )
    fig.show()    

In [636]:
# get resume
r  = get_resume_text('data/resume.docx',True)  # original
r1 = get_resume_text('data/resume_2023.docx',True) # best
r2 = get_resume_text('data/resume_2023.pdf',True) # bad
r3 = get_resume_text('data/resume2023.txt',True)  # no better


Contact

 Ken Constable, BSc. MSc. CFA

 ken.constable@gmail.com

 647-349-6500



1706-11 Charlotte St

Toronto

Ontario

M5V 0M6



Education

Master’s Degree in Data Science Msc.

Regis University

Data Science



Bachelor’s Degree in Chemistry Bsc.

University of Waterloo

Chemistry



CFA Charter holder

CFA Institute



Technical skills

Financial Models: Monte Carlo Simulations, back-testing, scenario-testing

Data Analysis: Python, Pandas, numpy, Scikit-learn, Tensorflow, Keras, R, Excel,VBA, xlwings, Beautiful Soup

Market Data: Bloomberg, Factset, BarraOne, Eagle, Alphavantage

Business Intelligence: SQL, SQL SERVER, ORACLE, HIVE, POSTGRES, SSIS, SSRS, ETL, ELT, MySQL, AWS Redshift, AWS S3, Airflow, AWS Glue

Machine Learning: NLP, Natural Language Processing, Deep Learning, Reinforcement Learning, Time-series Analysis, Supervised Learning, Unsupervised Learning,Regression, Classification, Decision Trees, Random Forest, XGBoost, Statistics, Optimization

Data Visualization: T

FileNotFoundError: [Errno 2] No such file or directory: 'data/resume_2023.docx'

In [596]:
# calc similarities with each job
df_r = calc_similarities(r,job_details,'data/terms.csv')
df_r

,title,company,location,similarity-score,keyword-coverage,keyword-wt,keywords-missing
10,Sr. Data Analyst,Ontario College Of Pharmacists Ordre des Pharm...,"Toronto, ON",0.373390,0.777778,5.666667,"{mathematics, matlab, computer science, modeli..."
1,Data Analyst,Brik Partners,"Toronto, ON",0.328218,0.750000,6.139535,"{matlab, metadata, data analysis skills, compu..."
21,Senior Data Analyst,Thentia,"Toronto, ON",0.263093,0.576923,4.139535,"{saas, ai, computer science, insights, big dat..."
23,Data Scientist,Focal Systems,"Toronto, ON",0.263087,0.750000,4.135135,"{ai, computer science, infrastructure, insight..."
8,Data Scientist,Clearco,"Toronto, ON",0.259482,0.666667,3.950820,"{modeling, ml, code, infrastructure, insurance..."
0,Data Scientist,Wish,"Toronto, ON",0.247380,0.888889,5.566667,"{mongodb, insights}"
4,Data Analyst,Connexall,"North York, ON",0.245884,0.793103,2.479167,"{workflow, computer science, insights, looker,..."
15,Data Scientist - Marketing Measurement & Optim...,ShyftLabs,"Toronto, ON",0.241822,0.731707,3.393939,"{communication skills, insights, looker, big d..."
14,"Data Scientist, Crypto Investment Firm",Recruiting From Scratch,"Toronto, ON",0.234655,0.869565,4.540541,"{git, insights, exploration}"
22,Data Science and Analytics Specialist,Healthcare Consultancy Group,"Toronto, ON",0.228518,0.680000,1.557377,"{snowflake, communication skills, pharma, comp..."


In [597]:
# plot similarities to resume
plot_similarity_scores(df_r)

In [637]:
df_r1 = calc_similarities(r1,job_details,'data/terms.csv')
df_r1

,title,company,location,similarity-score,keyword-coverage,keyword-wt,keywords-missing
10,Sr. Data Analyst,Ontario College Of Pharmacists Ordre des Pharm...,"Toronto, ON",0.381950,0.805556,5.277778,"{mathematics, matlab, computer science, infras..."
1,Data Analyst,Brik Partners,"Toronto, ON",0.324294,0.750000,5.441860,"{matlab, metadata, data analysis skills, compu..."
21,Senior Data Analyst,Thentia,"Toronto, ON",0.266259,0.576923,3.883721,"{saas, ai, computer science, insights, big dat..."
23,Data Scientist,Focal Systems,"Toronto, ON",0.261883,0.750000,3.702703,"{ai, computer science, infrastructure, insight..."
8,Data Scientist,Clearco,"Toronto, ON",0.261634,0.696970,3.491803,"{ml, infrastructure, code, insurance, data eng..."
0,Data Scientist,Wish,"Toronto, ON",0.249669,0.833333,5.033333,"{mongodb, master’s degree, insights}"
4,Data Analyst,Connexall,"North York, ON",0.248815,0.758621,1.958333,"{workflow, computer science, insights, looker,..."
15,Data Scientist - Marketing Measurement & Optim...,ShyftLabs,"Toronto, ON",0.239729,0.682927,2.772727,"{communication skills, regression, master’s de..."
14,"Data Scientist, Crypto Investment Firm",Recruiting From Scratch,"Toronto, ON",0.239557,0.782609,4.108108,"{regression, insights, clustering, exploration..."
24,VP of RevOps and Data Analytics,When I Work,"Toronto, ON",0.231973,0.750000,-0.363636,"{saas, infrastructure, insights, code, devops,..."


In [638]:
plot_similarity_scores(df_r1)

In [539]:
# get resume
cl = get_resume_text('data/cover_letter.docx',False)


# calc similarities with each job
df_cl = calc_similarities(cl,job_details,'data/terms.csv')
df_cl

,title,company,location,similarity-score,keyword-coverage,keyword-wt,keywords-missing
3,Sr. Data Analyst,Ontario College Of Pharmacists Ordre des Pharm...,"Toronto, ON",0.397530,0.250000,-1.037037,"{project management, python, data mining, infr..."
6,Data Scientist,Arup,"Toronto, ON",0.377605,0.268293,-2.208333,"{python, data mining, pipelines, ml, insights,..."
0,Data Scientist,Wish,"Toronto, ON",0.328269,0.388889,0.033333,"{data visualization, engineering, python, pipe..."
1,Data Science Co-op (R&D) - Summer term,Swift Medical,"Toronto, ON",0.309663,0.333333,-2.037736,"{predictive, ai, python, engineering, mentorin..."
9,Sr. IT Data Analyst,TD Bank,"Toronto, ON",0.306582,0.160000,-0.596491,"{data profiling, project management, metadata,..."
11,Senior Data Analyst,Sun Life,"Toronto, ON",0.288357,0.250000,-0.820000,"{python, insights, testing, sas, sql, options,..."
12,"Manager, Machine Learning and Advanced Analytics",MLSE,"Toronto, ON",0.260264,0.263158,-0.820896,"{project management, pipelines, insights, test..."
2,Data Analyst - Operations,Flair Airlines Ltd.,"Toronto, ON",0.255140,0.269231,-0.054054,"{data profiling, python, pipelines, insights, ..."
14,"Senior Product Manager, Growth Marketing & Ana...",Angi,"Toronto, ON",0.254424,0.350000,-1.400000,"{snowflake, engineering, features, project man..."
5,"Senior Data Scientist, Trust & Safety and Supp...",Thumbtack,"Toronto, ON",0.233622,0.322581,-0.360000,"{python, infrastructure, insights, sql, econom..."


In [577]:
# plot similarities to cover letter
plot_similarity_scores(df_cl)

In [639]:
# plot resume comparision to a specific job 
plot_keyword_comparison(r,job_details[0],'data/terms.csv')

In [541]:
# compare job summary and cover letter summary
summary_job = get_text_summary(job_details[14]['desc'],'text','lsa',3)
summary_cl = get_text_summary(cl,'text','lsa',3)

calc_similarity(summary_cl,summary_job)


Text Summary:Top 3 sentences using a lsa summarizer
Additionally, you will be responsible for developing and implementing the product roadmap for our enterprise analytics platform, and data pipelines and feeds necessary to facilitate those operations.
Develop and implement the product roadmap for our event-driven analytics platform, including features related to user profiles and targeting, martech activation and channel attribution.
Analyze and interpret marketing data to identify trends, patterns, and insights that can inform our product development roadmap.

Text Summary:Top 3 sentences using a lsa summarizer
I feel that this career opportunity offered with your organization is a strong match for my experience in data science, programming, and quantitative investment analysis.
As a Chartered Financial Analyst with a Master’s degree in Data Science I have over 20+ years of experience working on data-driven projects.
In my current role at Firmex, I am the principal data scientist buil

0.043894460814465706